<a href="https://colab.research.google.com/github/charann90/Scraping-Doctor-Information-Empowering-Healthcare-Analysis/blob/main/doctors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [3]:
import csv

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
df=pd.read_csv('/content/pedia1.csv')
len(df)

FileNotFoundError: ignored

In [46]:
df.drop(['Name', 'address', 'Phone_number'], axis=1, inplace=True)

In [47]:
df

,links
2,https://www.yellowpages.com/search?search_term...
3,https://www.yellowpages.com/search?search_term...
4,https://www.yellowpages.com/search?search_term...
5,https://www.yellowpages.com/search?search_term...
6,https://www.yellowpages.com/search?search_term...
...,...
865,https://www.yellowpages.com/search?search_term...
866,https://www.yellowpages.com/search?search_term...
867,https://www.yellowpages.com/search?search_term...
868,https://www.yellowpages.com/search?search_term...


In [27]:
links = df['links']  # Example: Process the first 20 links
batch_size = 5  # Number of links to process in each batch
num_processes = 10  # Number of processes to use

names = []
addresses = []
phone_numbers = []
personal_links = []
categories = []
years = []

# Define a function to process a batch of links
def process_batch(batch_links):
    batch_names = []
    batch_addresses = []
    batch_phone_numbers = []
    batch_personal_links = []
    batch_categories = []
    batch_years = []

    for link in batch_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the doctor's name
        name_element = soup.find("a", class_="business-name")
        name = name_element.text.split(', ')[0] if name_element else ""

        # Extract the address
        address_element = soup.find("div", class_="adr")
        address = address_element.get_text(strip=True) if address_element else ""

        # Extract the phone number
        phone_number_element = soup.find("div", class_="phones phone primary")
        phone_number = phone_number_element.text.strip() if phone_number_element else ""

        # Extract the personal page link
        try:
            personal_page_element = soup.find("div", class_="links")
            page = personal_page_element.find("a")["href"] if personal_page_element else ""
        except (AttributeError, TypeError):
            page = ""

        # Extract the categories
        cat_ele = soup.find("div", class_="categories")
        categories = [category.get_text(strip=True) for category in cat_ele.find_all("a")] if cat_ele else []

        # Extract the years of experience
        year_ele = soup.find("div", class_="number")
        year = year_ele.text if year_ele else ""

        batch_names.append(name)
        batch_addresses.append(address)
        batch_phone_numbers.append(phone_number)
        batch_personal_links.append(page)
        batch_categories.append(categories)
        batch_years.append(int(year) if year else 0)

    return (
        batch_names, batch_addresses, batch_phone_numbers,
        batch_personal_links, batch_categories, batch_years
    )

# Split the links into batches
batches = [links[i:i+batch_size] for i in range(0, len(links), batch_size)]

# Create a ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Submit tasks for each batch of links
    results = [executor.submit(process_batch, batch) for batch in batches]

    # Process the results as they complete
    for future in concurrent.futures.as_completed(results):
        # Retrieve the results of a completed task
        (
            batch_names, batch_addresses, batch_phone_numbers,
            batch_personal_links, batch_categories, batch_years
        ) = future.result()

        # Append the batch results to the respective lists
        names.extend(batch_names)
        addresses.extend(batch_addresses)
        phone_numbers.extend(batch_phone_numbers)
        personal_links.extend(batch_personal_links)
        categories.extend(batch_categories)
        years.extend(batch_years)

print("Names:", names)
print("Addresses:", addresses)
print("Phone Numbers:", phone_numbers)
print("Personal Links:", personal_links)
print("Categories:", categories)
print("Years:", years)


Names: ['', 'Advanced Urology', 'Dr. Yana Tavyev', 'Dr. Yana Tavyev', 'Dr. Philip H. Seifer', 'Benjamin Tiet', 'Benjamin Tiet', 'Dr. Yana Tavyev', 'Kaukauna Clinic', 'Benjamin Tiet', 'Advanced Urology', 'Dr. Lawrence Ian Kagan', 'Eisner Pediatric & Family Medical Center', 'California Kids Pediatrics', 'ABC Pediatrics Inc', 'California Kids Pediatrics', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Eisner Pediatric & Family Medical Center', 'Dr. Manpreet Virk', 'Eisner Pediatric & Family Medical Center', 'Christine Kao', 'Comprehensive Urology Medical Group', 'Kaukauna Clinic', 'Danelle M. Fisher', 'Danelle M. Fisher', 'Danelle M. Fisher', 'Benjamin Tiet', 'Eisner Pediatric & Family Medical Center', 'Advanced Urology', 'Charlotte Sarshad', 'Benjamin Tiet', 'Christine Kao', 'Advanced Urology', 'Charlotte Sarshad', 'Nicole K. Militello', 'Dr. Yovana Graciela Bruno', 'California Kids Pediatrics', 'Advanced Urology', 'Advanced Urology', 'Advanced Urol

In [41]:
df['links']=df['links'].iloc[1:]

len(df['links'])

870

In [49]:
df['Name']=names
df['Address']=addresses
df['Phone_number']= phone_numbers
df['Categories']= categories
df['Personal_website']=personal_links
df['Years_in_practice']= years

In [50]:
df

,Name,links,address,Phone_number,Address,Categories,Personal_website,Years_in_practice
0,,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",,,[],,0
1,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26
2,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","[Physicians & Surgeons, Pediatrics, Physicians...",https://www.cedars-sinai.edu,22
3,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","[Physicians & Surgeons, Pediatrics, Physicians...",https://www.cedars-sinai.edu,15
4,Dr. Philip H. Seifer,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(323) 857-2000,"6041 Cadillac Ave Ste 451Los Angeles, CA 90034","[Physicians & Surgeons, Pediatrics, Physicians...",https://healthy.kaiserpermanente.org/health/po...,30
...,...,...,...,...,...,...,...,...
865,Advanced Urology,https://www.yellowpages.com/search?search_term...,"1403 Lomita BlvdHarbor City, CA 90710",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26
866,Advanced Urology,https://www.yellowpages.com/search?search_term...,"8700 Beverly BlvdWest Hollywood, CA 90048",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26
867,Assal Hakhamjani,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 274-3260,"8525 W Pico BlvdLos Angeles, CA 90035","[Physicians & Surgeons, Pediatrics]",,30
868,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26


In [6]:
df=pd.read_csv('/content/pedia4.csv')

In [7]:
df

,Name,links,address,Phone_number,Address,Categories,Personal_website,Years_in_practice
0,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26
1,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","['Physicians & Surgeons, Pediatrics', 'Physici...",https://www.cedars-sinai.edu,22
2,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","['Physicians & Surgeons, Pediatrics', 'Physici...",https://www.cedars-sinai.edu,15
3,Dr. Philip H. Seifer,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(323) 857-2000,"6041 Cadillac Ave Ste 451Los Angeles, CA 90034","['Physicians & Surgeons, Pediatrics', 'Physici...",https://healthy.kaiserpermanente.org/health/po...,30
4,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(310) 825-5296,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...","['Physicians & Surgeons, Pediatrics', 'Physici...",NaN,22
...,...,...,...,...,...,...,...,...
864,Advanced Urology,https://www.yellowpages.com/search?search_term...,"1403 Lomita BlvdHarbor City, CA 90710",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26
865,Advanced Urology,https://www.yellowpages.com/search?search_term...,"8700 Beverly BlvdWest Hollywood, CA 90048",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26
866,Assal Hakhamjani,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 274-3260,"8525 W Pico BlvdLos Angeles, CA 90035","['Physicians & Surgeons, Pediatrics']",NaN,30
867,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26


In [51]:
df.to_csv('pedia4.csv',index=False)

In [5]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

In [ ]:
import csv

In [5]:
len(links)

870

In [8]:
import pandas as pd
import concurrent.futures

In [9]:
import multiprocessing

In [79]:
df=pd.DataFrame({'Name':names,'links':links,'address':addresses,'Phone_number':phone_numbers,'Personal_page':personal_page})

In [21]:
df=pd.read_csv('/content/pedia1.csv')

In [51]:
df.to_csv('pedia3.csv',index=False)

1. Seriel Number
2. Base Link from the Yellow Pages
3. Doctors/Business Name
4. Individual Profile Link fron the Yellow Pages
5. Categories
6. Contact Number
7. Fax Number
8. Email Id
9. Address
10. City
11. Years in Business
12. Individual Website Link
13. Facebook
14. Instagram
15. Twitter
16. Youtube
17. LinkdIn

In [10]:
print(len(years))

870


In [9]:
import re

In [11]:
facebook_links = []

for personal_page in df['Personal_website'][:4]:
    if personal_page:
        response = requests.get(personal_page)
        soup = BeautifulSoup(response.content, "html.parser")

        facebook_link_element = soup.find('a', href=re.compile(r'facebook.com'))
        facebook_link = facebook_link_element['href'] if facebook_link_element else ""
        facebook_links.append(facebook_link)
    else:
        facebook_links.append("")
print(facebook_links)

['https://www.facebook.com/Advanced-Urology-Medical-Offices-102490307836987/', 'https://www.facebook.com/CedarsSinai', 'https://www.facebook.com/CedarsSinai', 'https://www.facebook.com/kpthrive']


In [13]:
linkedin_links = []

for personal_page in df['Personal_website'][:4]:
    if personal_page:
        response = requests.get(personal_page)
        soup = BeautifulSoup(response.content, "html.parser")

        linkedin_link_element = soup.find('a', href=re.compile(r'linkedin.com'))
        linkedin_link = linkedin_link_element['href'] if linkedin_link_element else ""
        linkedin_links.append(linkedin_link)
    else:
        linkedin_links.append("")
        
print(linkedin_links)

['https://www.linkedin.com/company/advanced-urology', 'https://www.linkedin.com/company/cedars-sinai-medical-center/mycompany/', 'https://www.linkedin.com/company/cedars-sinai-medical-center/mycompany/', '']


In [16]:
import re

instagram_links = []
email_addresses = []
twitter_links = []

for personal_page in df['Personal_website'][:5]:
    if personal_page:
        response = requests.get(personal_page)
        soup = BeautifulSoup(response.content, "html.parser")

        # Instagram link
        instagram_link_element = soup.find('a', href=re.compile(r'instagram.com'))
        instagram_link = instagram_link_element['href'] if instagram_link_element else ""
        instagram_links.append(instagram_link)

        # Email address
        email_element = soup.find('a', href=re.compile(r'mailto:'))
        email_address = email_element['href'].replace('mailto:', '') if email_element else ""
        email_addresses.append(email_address)

        # Twitter link
        twitter_link_element = soup.find('a', href=re.compile(r'twitter.com'))
        twitter_link = twitter_link_element['href'] if twitter_link_element else ""
        twitter_links.append(twitter_link)
    else:
        instagram_links.append("")
        email_addresses.append("")
        twitter_links.append("")

print("Instagram Links:", instagram_links)
print("Email Addresses:", email_addresses)
print("Twitter Links:", twitter_links)


MissingSchema: ignored

In [19]:
instagram_links = []
email_addresses = []
twitter_links = []
facebook_links = []
linkedin_links = []

for personal_page in df['Personal_website']:
    if personal_page and not pd.isna(personal_page):
        try:
            response = requests.get(personal_page)
            soup = BeautifulSoup(response.content, "html.parser")

            # Instagram link
            instagram_link_element = soup.find('a', href=re.compile(r'instagram.com'))
            instagram_link = instagram_link_element['href'] if instagram_link_element else ""
            instagram_links.append(instagram_link)

            # Email address
            email_element = soup.find('a', href=re.compile(r'mailto:'))
            email_address = email_element['href'].replace('mailto:', '') if email_element else ""
            email_addresses.append(email_address)

            # Twitter link
            twitter_link_element = soup.find('a', href=re.compile(r'twitter.com'))
            twitter_link = twitter_link_element['href'] if twitter_link_element else ""
            twitter_links.append(twitter_link)

            # Facebook link
            facebook_link_element = soup.find('a', href=re.compile(r'facebook.com'))
            facebook_link = facebook_link_element['href'] if facebook_link_element else ""
            facebook_links.append(facebook_link)

            # LinkedIn link
            linkedin_link_element = soup.find('a', href=re.compile(r'linkedin.com'))
            linkedin_link = linkedin_link_element['href'] if linkedin_link_element else ""
            linkedin_links.append(linkedin_link)
        except (requests.exceptions.RequestException, ValueError):
            instagram_links.append("")
            email_addresses.append("")
            twitter_links.append("")
            facebook_links.append("")
            linkedin_links.append("")
    else:
        instagram_links.append("")
        email_addresses.append("")
        twitter_links.append("")
        facebook_links.append("")
        linkedin_links.append("")

print("Instagram Links:",instagram_links )
print("Email Addresses:", email_addresses)
print("Twitter Links:", twitter_links)
print("Facebook Links:", facebook_links)
print("LinkedIn Links:", linkedin_links)


Instagram Links: ['', 'https://www.instagram.com/cedarssinai/', 'https://www.instagram.com/cedarssinai/', 'https://www.instagram.com/kpthrive/', '', '', 'https://www.instagram.com/cedarssinai/', '', '', '', '', '', 'https://www.instagram.com/cakidsmd/', '//www.instagram.com/', 'https://www.instagram.com/cakidsmd/', '', '', '', '', '', '', '', '', 'https://www.instagram.com/optum/?hl=en', '', '', '', '', '', '', '', '', '', '', 'https://www.instagram.com/optum/?hl=en', '', '', 'https://instagram.com/uclahealth', '', 'https://www.instagram.com/cakidsmd/', '', '', '', '', 'https://www.instagram.com/cakidsmd/', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'https://www.instagram.com/blvdpeds/', '', '', '', '', '', '', '', '', '', '', 'https://www.instagram.com/adventisthealth/', 'https://www.instagram.com/adventisthealth/', '', '', '', '', '', 'https://www.instagram.com/adventisthealth/', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'htt

In [20]:
df['Email']= email_addresses
df['Facebook']=facebook_links
df['LinkedIn']=linkedin_links
df['Twitter']=twitter_links
df['Instagram']=instagram_links

In [21]:
df

,Name,links,address,Phone_number,Address,Categories,Personal_website,Years_in_practice,Email,Facebook,LinkedIn,Twitter,Instagram
0,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26,Katee@UrologyLA.com,https://www.facebook.com/Advanced-Urology-Medi...,https://www.linkedin.com/company/advanced-urology,https://twitter.com/UrologyMedical,
1,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","['Physicians & Surgeons, Pediatrics', 'Physici...",https://www.cedars-sinai.edu,22,,https://www.facebook.com/CedarsSinai,https://www.linkedin.com/company/cedars-sinai-...,https://twitter.com/cedarssinai,https://www.instagram.com/cedarssinai/
2,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","['Physicians & Surgeons, Pediatrics', 'Physici...",https://www.cedars-sinai.edu,15,,https://www.facebook.com/CedarsSinai,https://www.linkedin.com/company/cedars-sinai-...,https://twitter.com/cedarssinai,https://www.instagram.com/cedarssinai/
3,Dr. Philip H. Seifer,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(323) 857-2000,"6041 Cadillac Ave Ste 451Los Angeles, CA 90034","['Physicians & Surgeons, Pediatrics', 'Physici...",https://healthy.kaiserpermanente.org/health/po...,30,,https://www.facebook.com/kpthrive,,https://twitter.com/kpthrive,https://www.instagram.com/kpthrive/
4,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(310) 825-5296,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...","['Physicians & Surgeons, Pediatrics', 'Physici...",NaN,22,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,Advanced Urology,https://www.yellowpages.com/search?search_term...,"1403 Lomita BlvdHarbor City, CA 90710",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26,Katee@UrologyLA.com,https://www.facebook.com/Advanced-Urology-Medi...,https://www.linkedin.com/company/advanced-urology,https://twitter.com/UrologyMedical,
865,Advanced Urology,https://www.yellowpages.com/search?search_term...,"8700 Beverly BlvdWest Hollywood, CA 90048",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26,Katee@UrologyLA.com,https://www.facebook.com/Advanced-Urology-Medi...,https://www.linkedin.com/company/advanced-urology,https://twitter.com/UrologyMedical,
866,Assal Hakhamjani,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 274-3260,"8525 W Pico BlvdLos Angeles, CA 90035","['Physicians & Surgeons, Pediatrics']",NaN,30,,,,,
867,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","['Physicians & Surgeons, Pediatrics', 'Medical...",http://www.advancedurology.net,26,Katee@UrologyLA.com,https://www.facebook.com/Advanced-Urology-Medi...,https://www.linkedin.com/company/advanced-urology,https://twitter.com/UrologyMedical,


In [22]:
df.to_csv('final_pedia.csv',index=False)